In [ ]:
!pip uninstall -y pyarrow datasets
!pip install --no-use-pep517 pyarrow
!pip install datasets
!pip install torch transformers
!pip install transformers[torch]
!pip install accelerate -U
!pip install scikit-learn

In [ ]:
import os

import pandas as pd
import s3fs
import zipfile

## Récupérer les données d'un challenge

In [ ]:
# Create filesystem object
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})

In [ ]:
# Lister les fichiers d'un challenge
fs.ls("civel/diffusion/hackathon-minarm-2024/AIVSAI/hack_train.csv")

In [ ]:
# Télécharger les données dans le service
PATH_IN = 'civel/diffusion/hackathon-minarm-2024/AIVSAI/hack_train.csv'
fs.download(PATH_IN, 'data/hack_train.csv')

In [ ]:
df = pd.read_csv('data/hack_train.csv')

In [ ]:
df

## Model

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

# Create a custom dataset class
class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]

# Define the feedforward neural network model
class FeedforwardModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(FeedforwardModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# Assuming you have your data in 'df' DataFrame
texts = df['text'].tolist()
labels = df['label'].tolist()

# Convert texts to numerical representations using bag-of-words
vectorizer = CountVectorizer()
input_data = vectorizer.fit_transform(texts).toarray()

In [ ]:
# Split the data into train and test sets
train_data, test_data, train_labels, test_labels = train_test_split(input_data, labels, test_size=0.2)

# Create PyTorch datasets
train_dataset = TextDataset(train_data, train_labels)
test_dataset = TextDataset(test_data, test_labels)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)


In [26]:
# Define the model, loss function, and optimizer
input_size = len(input_data[0])  # Size of the input vector
hidden_size = 128
output_size = len(set(labels))  # Number of unique labels
model = FeedforwardModel(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for batch_data, batch_labels in train_loader:
        # Forward pass
        outputs = model(batch_data)
        loss = criterion(outputs, batch_labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

In [ ]:
# Evaluation
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for batch_data, batch_labels in test_loader:
        outputs = model(batch_data)
        _, predicted = torch.max(outputs.data, 1)
        total += batch_labels.size(0)
        correct += (predicted == batch_labels).sum().item()

    accuracy = correct / total
    print(f"Accuracy: {accuracy:.4f}")

In [ ]:
!nvidia-smi

In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score
import torch
import torch.cuda

print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())

# Split the dataset
train_test_split = tokenized_datasets.train_test_split(test_size=0.2, seed=42)
train_dataset = train_test_split['train']
test_val_dataset = train_test_split['test']

test_val_split = test_val_dataset.train_test_split(test_size=0.5, seed=42)
eval_dataset = test_val_split['train']
test_dataset = test_val_split['test']

# Load the model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
print(model.device)

# Check for GPU availability and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(model.device)

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=100
)

# Define the metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='binary')
    return {
        'accuracy': acc,
        'f1_score': f1,
    }

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics
)

# Start training
trainer.train()

# Save the trained model
torch.save(model.state_dict(), 'model_dataset2.pth')

In [ ]:
# Set the model to evaluation mode
model.eval()

# Move the model to the appropriate device (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
# Evaluate the model on the test dataset
predictions = trainer.predict(test_dataset)
test_metrics = compute_metrics(predictions)
print("Test Accuracy:", test_metrics['accuracy'])
print("Test F1 Score:", test_metrics['f1_score'])

In [ ]:
import torch
from torch.nn.functional import softmax

# Check for GPU availability and set the device accordingly
device = torch.device("cpu" if torch.cuda.is_available() else "cuda")
model.to(device)  # Move the model to the appropriate device

# Function to make a prediction on a single sentence
def predict(sentence):
    # Tokenize the sentence so it matches the format expected by the model
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # Predict
    with torch.no_grad():  # Disable gradient calculation to speed up the process and reduce memory usage
        logits = model(**inputs).logits

    # Apply softmax to logits to get probabilities
    probabilities = softmax(logits, dim=1)

    # Assuming we have two classes, 0 and 1, and class 0 is the 'negative' class
    prediction = probabilities.argmax().item()  # Get the index of the highest probability
    return {"class": prediction, "probabilities": probabilities.tolist()[0]}

# Example usage
user_sentence = "Yes, it is possible to be subject to a cash withdrawal even if you do not use an ATM. There are several ways that this could happen:Debit card transactions: If you make a purchase using your debit card, the merchant may automatically withdraw the amount of the purchase from your checking account. This is essentially the same as making a cash withdrawal.Bank fees: Some banks charge fees for maintaining an account or for using certain services. These fees may be automatically withdrawn from your account on a regular basis.Automatic payments: If you have set up automatic payments for bills or other expenses, the amount of the payment will be withdrawn from your account when it is due.Check payments: If you write a check to pay for something, the recipient may deposit the check and withdraw the funds from your account.Electronic transfers: You may also be subject to a cash withdrawal if you authorize an electronic transfer of funds from your account to another account.In summary, there are many ways that you could be subject to a cash withdrawal even if you do not use an ATM. It is important to carefully track your account balance and be aware of any automatic transactions or payments that may be taking place."
result = predict(user_sentence)
print("Predicted Class:", result["class"])
print("Probabilities:", result["probabilities"])